In [10]:
from mGST import compatibility,algorithm, optimization, low_level_jit, additional_fns
from mGST.reporting import uncertainty, reporting

import pickle as pickle
from pygsti.report import reportables as rptbl #Needs cvxpy!
from pygsti.modelpacks import smq1Q_XYI as std
import pygsti
from argparse import Namespace
from itertools import product

import numpy as np
import pandas as pd
import numpy.linalg as la
import json
import pandas
import matplotlib.pyplot as plt
from os import listdir
from exp_design.target_gates import *

In [11]:
folder_name = "exp_data"
filenames = listdir(path = folder_name)
datafile_names  = [s for s in filenames if ".txt" in s]

In [13]:
# Import sequence list
J_list = pandas.read_csv("exp_design/sequences.csv", delimiter=",", names = list(range(32))).values
N = J_list.shape[0] + 1
J_list = [[int(x) for x in J_list[i,:] if str(x) != 'nan'] for i in range(N-1)]
J_list.insert(0,[])
l_max = np.max([len(J_list[i]) for i in range(N)])

J = []
for i in range(N):
    J.append(list(np.pad(J_list[i],(0,l_max-len(J_list[i])),'constant',constant_values=-1)))
J = np.array(J).astype(int)[:,::-1]

In [ ]:
max_samples = 2000 # Maximum number of samples per sequence and per file
data = np.zeros((len(J),max_samples+1))
for filename in datafile_names:
    print(filename)
    if filename == datafile_names[0]:
        data_file = pandas.read_csv(folder_name + "/" + filename, sep = r',|: ', engine = 'python', dtype = str, names = list(range(max_samples)))
        data_file.set_index(0, inplace=True)
    else:
        data_file_new = pandas.read_csv(folder_name + "/" + filename, sep = r',|: ', engine = 'python', dtype = str, names = list(range(max_samples)))
        data_file_new.set_index(0, inplace=True)
        data_file = pandas.concat([data_file,data_file_new], axis = 1, join = 'outer')
        data_file.dropna(axis = 1, how = 'all', inplace = True)
    #data_file = data_file.drop(data_file.columns[0], axis=1)
    # data[:len(data_file),:] = data_file
data_file.sort_values(by = 0, axis = 0, key = np.int32, inplace = True)

In [ ]:
basis_dict = {'00':0, '01':0, '10':1, '11':1}
# zero_fidelity = 0.995

y = np.zeros((pdim,N))
sample_counts = []
for i in range(N):
    valid_samples = (data_file.values[i,:][~np.isnan(data_file.values[i,:].astype(float))])[:180] ### !!! Don't forget to change this back !!!
    for k in range(len(valid_samples)): 
        j = basis_dict[valid_samples[k]]
        y[j,i] += 1

    y[:,i] = y[:,i]/(len(valid_samples))
    sample_counts.append(len(valid_samples))
    
meas_samples = int(np.average(sample_counts))
print(meas_samples)